# Quality Dashboard

## Purpose

This notebook teaches you how to create interactive quality dashboards for real-time and historical quality monitoring. You'll learn to build dashboards with time series charts, quality maps, metrics grids, and trend analysis.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Create quality dashboards with interactive widgets
- ✅ Monitor quality metrics in real-time
- ✅ Analyze quality trends over time
- ✅ Generate quality reports
- ✅ Set up alerts and notifications

## Estimated Duration

45-60 minutes

---

## Overview

Quality dashboards provide real-time and historical views of data quality metrics. The AM-QADF framework supports:

- 📊 **Real-time Monitoring**: Live quality metrics updates
- 📈 **Historical Analysis**: Quality trends over time
- 🔄 **Comparison Mode**: Compare different builds or time periods
- 📋 **Metrics Grid**: Multiple quality metrics at a glance
- ⚠️ **Alert System**: Automatic alerts for quality issues

Use the interactive widgets below to build and customize quality dashboards - no coding required!


In [ ]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from typing import Optional, Tuple, Dict, Any, List
import time

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import dashboard classes
DASHBOARD_AVAILABLE = False
try:
    from am_qadf.analytics.reporting.visualization import QualityDashboardGenerator
    DASHBOARD_AVAILABLE = True
    print("✅ Quality Dashboard Generator available")
except ImportError as e:
    # QualityDashboardGenerator will be created if needed, but for now we use MongoDB directly
    DASHBOARD_AVAILABLE = False
    print(f"ℹ️ Dashboard generator not available: {e} - using direct MongoDB access")

# MongoDB connection setup
INFRASTRUCTURE_AVAILABLE = False
mongo_client = None
voxel_storage = None
stl_client = None

try:
    from src.infrastructure.config import MongoDBConfig
    from src.infrastructure.database import MongoDBClient
    from am_qadf.voxel_domain import VoxelGridStorage
    from am_qadf.query import STLModelClient
    
    # Initialize MongoDB connection
    config = MongoDBConfig.from_env()
    if not config.username:
        config.username = os.getenv('MONGO_ROOT_USERNAME', 'admin')
    if not config.password:
        config.password = os.getenv('MONGO_ROOT_PASSWORD', 'password')
    
    mongo_client = MongoDBClient(config=config)
    if mongo_client.is_connected():
        voxel_storage = VoxelGridStorage(mongo_client=mongo_client)
        stl_client = STLModelClient(mongo_client=mongo_client)
        INFRASTRUCTURE_AVAILABLE = True
        print(f"✅ Connected to MongoDB: {config.database}")
    else:
        print("⚠️ MongoDB connection failed")
except Exception as e:
    print(f"⚠️ MongoDB not available: {e} - using demo mode")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ Quality Dashboard Generator available
✅ Connected to MongoDB: am_qadf_data
✅ Setup complete!


## Interactive Quality Dashboard

Use the widgets below to create and customize quality dashboards. Select dashboard mode, configure metrics, and visualize quality trends interactively!


In [ ]:
# Create Interactive Quality Dashboard

# Global state
dashboard_data = {}
time_series_data = {}
current_timestamp = datetime.now()
current_model_id = None
selected_grids = []
quality_assessments_data = []

# ============================================
# Helper Functions for Demo Data
# ============================================

def generate_time_series_quality_data(start_time, end_time, interval_minutes=5):
    """Generate sample time series quality data."""
    np.random.seed(42)
    
    # Create time range
    time_range = pd.date_range(start=start_time, end=end_time, freq=f'{interval_minutes}min')
    
    # Generate quality metrics with trends
    n_points = len(time_range)
    
    # Overall quality (trending slightly down)
    base_quality = 0.85
    trend = np.linspace(0, -0.1, n_points)
    noise = np.random.normal(0, 0.02, n_points)
    overall_quality = np.clip(base_quality + trend + noise, 0.0, 1.0)
    
    # Completeness (stable with small variations)
    completeness = np.clip(0.90 + np.random.normal(0, 0.03, n_points), 0.0, 1.0)
    
    # Coverage (slight upward trend)
    coverage_base = 0.80
    coverage_trend = np.linspace(0, 0.05, n_points)
    coverage = np.clip(coverage_base + coverage_trend + np.random.normal(0, 0.02, n_points), 0.0, 1.0)
    
    # Consistency (stable)
    consistency = np.clip(0.88 + np.random.normal(0, 0.02, n_points), 0.0, 1.0)
    
    # SNR (decreasing trend)
    snr_base = 25.0
    snr_trend = np.linspace(0, -3, n_points)
    snr = np.clip(snr_base + snr_trend + np.random.normal(0, 1.0, n_points), 0.0, 100.0)
    
    return pd.DataFrame({
        'timestamp': time_range,
        'overall_quality': overall_quality,
        'completeness': completeness,
        'coverage': coverage,
        'consistency': consistency,
        'snr': snr
    })

# ============================================
# Top Panel: Dashboard Mode and Controls
# ============================================

# Data source mode
data_source_label = widgets.HTML("<b>Data Source:</b>")
data_source_mode = RadioButtons(
    options=[('MongoDB', 'mongodb'), ('Sample Data', 'sample')],
    value='mongodb',
    description='Source:',
    style={'description_width': 'initial'}
)

dashboard_mode = RadioButtons(
    options=[
        ('Real-time', 'realtime'),
        ('Historical', 'historical'),
        ('Comparison', 'comparison')
    ],
    value='historical',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Model selection (for MongoDB)
model_label = widgets.HTML("<b>Model:</b>")
model_options = [("━━━ Select Model ━━━", None)]
if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", m.get('model_id'))
            for m in models
        ])
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")

model_selector = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

# Grid type filter
grid_type_label = widgets.HTML("<b>Grid Type:</b>")
grid_type_filter = Dropdown(
    options=[
        ('All Grids', 'all'),
        ('Fused', 'fused'),
        ('Corrected', 'corrected'),
        ('Processed', 'processed'),
        ('Signal-Mapped', 'signal_mapped'),
        ('Raw', 'raw')
    ],
    value='fused',  # Default to fused grids
    description='Type:',
    style={'description_width': 'initial'}
)

# Time range selectors (for historical mode)
time_start_label = widgets.HTML("<b>Start:</b>")
time_start = Text(
    value=(datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d %H:%M'),
    description='Start:',
    style={'description_width': 'initial'},
    layout=Layout(width='150px')
)

time_end_label = widgets.HTML("<b>End:</b>")
time_end = Text(
    value=datetime.now().strftime('%Y-%m-%d %H:%M'),
    description='End:',
    style={'description_width': 'initial'},
    layout=Layout(width='150px')
)

refresh_button = Button(
    description='Refresh',
    button_style='success',
    icon='refresh',
    layout=Layout(width='120px')
)

auto_refresh = Checkbox(value=False, description='Auto-refresh', style={'description_width': 'initial'})
refresh_interval = IntSlider(value=60, min=10, max=300, step=10, description='Interval (s):', style={'description_width': 'initial'})

top_panel = VBox([
    HBox([data_source_label, data_source_mode, dashboard_mode]),
    HBox([model_label, model_selector, grid_type_label, grid_type_filter]),
    HBox([
        time_start_label,
        time_start,
        time_end_label,
        time_end,
        refresh_button
    ]),
    HBox([
        auto_refresh,
        refresh_interval
    ])
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Dashboard Configuration
# ============================================

# Metrics to Display
metrics_label = widgets.HTML("<b>Metrics to Display:</b>")
metric_data_quality = Checkbox(value=True, description='Data Quality', style={'description_width': 'initial'})
metric_signal_quality = Checkbox(value=True, description='Signal Quality', style={'description_width': 'initial'})
metric_alignment = Checkbox(value=False, description='Alignment Accuracy', style={'description_width': 'initial'})
metric_completeness = Checkbox(value=True, description='Completeness', style={'description_width': 'initial'})
metric_custom = Checkbox(value=False, description='Custom Metrics', style={'description_width': 'initial'})

metrics_section = VBox([
    metrics_label,
    metric_data_quality,
    metric_signal_quality,
    metric_alignment,
    metric_completeness,
    metric_custom
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Visualization Options
viz_options_label = widgets.HTML("<b>Visualization Options:</b>")
chart_type = Dropdown(
    options=[('Line', 'line'), ('Bar', 'bar'), ('Scatter', 'scatter'), ('Heatmap', 'heatmap')],
    value='line',
    description='Chart Type:',
    style={'description_width': 'initial'}
)
time_granularity = Dropdown(
    options=[('Second', 'second'), ('Minute', 'minute'), ('Hour', 'hour'), ('Day', 'day')],
    value='minute',
    description='Granularity:',
    style={'description_width': 'initial'}
)
aggregation = Dropdown(
    options=[('Mean', 'mean'), ('Max', 'max'), ('Min', 'min'), ('Latest', 'latest')],
    value='mean',
    description='Aggregation:',
    style={'description_width': 'initial'}
)

viz_options_section = VBox([
    viz_options_label,
    chart_type,
    time_granularity,
    aggregation
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Filters
filters_label = widgets.HTML("<b>Filters:</b>")
signal_filter = Dropdown(
    options=[('All', 'all'), ('Temperature', 'temperature'), ('Power', 'power'), ('Density', 'density')],
    value='all',
    description='Signal:',
    style={'description_width': 'initial'}
)
layer_min = IntSlider(value=0, min=0, max=100, step=1, description='Layer Min:', style={'description_width': 'initial'})
layer_max = IntSlider(value=100, min=0, max=100, step=1, description='Layer Max:', style={'description_width': 'initial'})
quality_threshold = FloatSlider(value=0.7, min=0.0, max=1.0, step=0.05, description='Quality Threshold:', style={'description_width': 'initial'})

filters_section = VBox([
    filters_label,
    signal_filter,
    layer_min,
    layer_max,
    quality_threshold
], layout=Layout(padding='5px', border='1px solid #ddd'))

left_panel = VBox([
    metrics_section,
    viz_options_section,
    filters_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Dashboard Visualizations
# ============================================

# Main quality chart
main_chart_output = Output(layout=Layout(height='300px', overflow='auto'))

# Quality map (for 3D visualization)
quality_map_output = Output(layout=Layout(height='200px', overflow='auto'))

# Metrics grid
metrics_grid_output = Output(layout=Layout(height='200px', overflow='auto'))

# Comparison view (when in comparison mode)
comparison_output = Output(layout=Layout(height='300px', overflow='auto'))

center_panel = VBox([
    widgets.HTML("<h3>Quality Dashboard</h3>"),
    widgets.HTML("<b>Main Quality Chart:</b>"),
    main_chart_output,
    widgets.HTML("<b>Quality Map:</b>"),
    quality_map_output,
    widgets.HTML("<b>Metrics Grid:</b>"),
    metrics_grid_output,
    widgets.HTML("<b>Comparison View:</b>"),
    comparison_output
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Dashboard Statistics
# ============================================

# Current Status
current_status_label = widgets.HTML("<b>Current Status:</b>")
overall_score_display = widgets.HTML("<h2 style='color: green;'>0.85</h2>")
status_indicator = widgets.HTML("<p style='color: green;'>✅ Pass</p>")
last_updated = widgets.HTML(f"<p><b>Last Updated:</b> {datetime.now().strftime('%H:%M:%S')}</p>")

current_status_section = VBox([
    current_status_label,
    overall_score_display,
    status_indicator,
    last_updated
], layout=Layout(padding='5px', border='2px solid #4CAF50'))

# Trend Analysis
trend_label = widgets.HTML("<b>Trend Analysis:</b>")
trend_direction = widgets.HTML("<p><b>Direction:</b> <span style='color: orange;'>Down</span></p>")
trend_magnitude = widgets.HTML("<p><b>Magnitude:</b> -0.05</p>")
trend_period = widgets.HTML("<p><b>Period:</b> 7 days</p>")

trend_section = VBox([
    trend_label,
    trend_direction,
    trend_magnitude,
    trend_period
], layout=Layout(padding='5px'))

# Alert Summary
alert_label = widgets.HTML("<b>Alert Summary:</b>")
alert_count = widgets.HTML("<p><b>Active Alerts:</b> <span style='color: orange;'>2</span></p>")
alert_list = widgets.HTML("<ul><li>⚠️ Quality below threshold</li><li>⚠️ SNR decreasing</li></ul>")
alert_severity = widgets.HTML("<p><b>Severity:</b> <span style='color: orange;'>Medium</span></p>")

alert_section = VBox([
    alert_label,
    alert_count,
    alert_list,
    alert_severity
], layout=Layout(padding='5px'))

# Quick Actions
actions_label = widgets.HTML("<b>Quick Actions:</b>")
generate_report_button = Button(description='Generate Report', button_style='', layout=Layout(width='150px'))
export_data_button = Button(description='Export Data', button_style='', layout=Layout(width='150px'))
set_alerts_button = Button(description='Set Alerts', button_style='', layout=Layout(width='150px'))
configure_dashboard_button = Button(description='Configure', button_style='', layout=Layout(width='150px'))

actions_section = VBox([
    actions_label,
    generate_report_button,
    export_data_button,
    set_alerts_button,
    configure_dashboard_button
], layout=Layout(padding='5px'))

right_panel = VBox([
    current_status_section,
    trend_section,
    alert_section,
    actions_section
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Status and Alerts
# ============================================

status_display = widgets.HTML("<b>Status:</b> Dashboard active")
last_refresh_display = widgets.HTML(f"<b>Last Refresh:</b> {datetime.now().strftime('%H:%M:%S')}")
alert_display = widgets.HTML("<b>Alerts:</b> 2 active warnings")
progress_indicator = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='info',
    layout=Layout(width='100%', display='none')
)

bottom_panel = VBox([
    HBox([status_display, last_refresh_display, alert_display]),
    progress_indicator
], layout=Layout(padding='10px', border='1px solid #ccc'))

# ============================================
# Dashboard Functions
# ============================================

def load_quality_assessments_from_mongodb(model_id=None, grid_type='fused', start_time=None, end_time=None):
    """Load quality assessments from MongoDB."""
    global quality_assessments_data
    
    if not voxel_storage or not mongo_client:
        return []
    
    try:
        # Get grids for the model
        if model_id:
            grids = voxel_storage.list_grids(model_id=model_id, limit=100)
        else:
            # Get all grids
            grids = list(voxel_storage.mongo_client.db['voxel_grids'].find({}, limit=100))
        
        assessments = []
        
        for grid in grids:
            metadata = grid.get('metadata', {})
            config_meta = metadata.get('configuration_metadata', {})
            if not config_meta:
                config_meta = metadata
            
            # Filter by grid type
            is_fused = config_meta.get('fusion_applied', False)
            is_corrected = config_meta.get('correction_applied', False)
            is_processed = config_meta.get('processing_applied', False)
            has_signals = len(grid.get('available_signals', [])) > 0
            
            grid_type_match = False
            if grid_type == 'all':
                grid_type_match = True
            elif grid_type == 'fused' and is_fused:
                grid_type_match = True
            elif grid_type == 'corrected' and is_corrected:
                grid_type_match = True
            elif grid_type == 'processed' and is_processed:
                grid_type_match = True
            elif grid_type == 'signal_mapped' and has_signals and not is_corrected and not is_processed and not is_fused:
                grid_type_match = True
            elif grid_type == 'raw' and not has_signals:
                grid_type_match = True
            
            if not grid_type_match:
                continue
            
            # Get quality assessments
            quality_assessments = metadata.get('quality_assessments', [])
            if not quality_assessments:
                # Check for latest quality assessment
                latest_assessment = metadata.get('latest_quality_assessment')
                if latest_assessment:
                    quality_assessments = [latest_assessment]
            
            for assessment in quality_assessments:
                assessment_timestamp_str = assessment.get('assessment_timestamp')
                if not assessment_timestamp_str:
                    continue
                
                try:
                    # Parse timestamp
                    if isinstance(assessment_timestamp_str, str):
                        assessment_timestamp = datetime.fromisoformat(assessment_timestamp_str.replace('Z', '+00:00'))
                    else:
                        assessment_timestamp = assessment_timestamp_str
                    
                    # Filter by time range
                    if start_time and assessment_timestamp < start_time:
                        continue
                    if end_time and assessment_timestamp > end_time:
                        continue
                    
                    # Extract metrics
                    metrics = assessment.get('metrics', {})
                    data_metrics = metrics.get('data', {})
                    signal_metrics = metrics.get('signal', {})
                    alignment_metrics = metrics.get('alignment', {})
                    completeness_metrics = metrics.get('completeness', {})
                    
                    assessments.append({
                        'timestamp': assessment_timestamp,
                        'grid_id': grid.get('grid_id', str(grid.get('_id', ''))),
                        'grid_name': grid.get('grid_name', 'Unknown'),
                        'model_id': grid.get('model_id', ''),
                        'model_name': grid.get('model_name', 'Unknown'),
                        'overall_score': assessment.get('overall_score', 0.0),
                        'completeness': data_metrics.get('completeness', 0.0),
                        'coverage': data_metrics.get('coverage_spatial', 0.0),
                        'consistency': data_metrics.get('consistency', 0.0),
                        'accuracy': data_metrics.get('accuracy', 0.0),
                        'reliability': data_metrics.get('reliability', 0.0),
                        'snr': signal_metrics.get('snr', 0.0),
                        'uncertainty': signal_metrics.get('uncertainty', 0.0),
                        'confidence': signal_metrics.get('confidence', 0.0),
                        'alignment_accuracy': alignment_metrics.get('spatial_accuracy', 0.0),
                        'gap_count': completeness_metrics.get('gap_count', 0),
                        'gap_percentage': completeness_metrics.get('gap_percentage', 0.0)
                    })
                except Exception as e:
                    # Skip invalid assessments
                    continue
        
        # Sort by timestamp
        assessments.sort(key=lambda x: x['timestamp'])
        return assessments
        
    except Exception as e:
        print(f"⚠️ Error loading quality assessments: {e}")
        return []

def refresh_dashboard(button=None):
    """Refresh dashboard data and visualizations."""
    global dashboard_data, time_series_data, current_timestamp, current_model_id, quality_assessments_data
    
    status_display.value = "<b>Status:</b> Refreshing dashboard..."
    progress_indicator.layout.display = 'flex'
    progress_indicator.value = 0
    
    try:
        if data_source_mode.value == 'mongodb':
            # Load from MongoDB
            if not model_selector.value:
                status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ Please select a model</span>"
                progress_indicator.layout.display = 'none'
                return
            
            current_model_id = model_selector.value
            grid_type = grid_type_filter.value
            
            # Parse time range
            start_time = datetime.strptime(time_start.value, '%Y-%m-%d %H:%M') if time_start.value else None
            end_time = datetime.strptime(time_end.value, '%Y-%m-%d %H:%M') if time_end.value else None
            
            progress_indicator.value = 20
            
            # Load quality assessments
            quality_assessments_data = load_quality_assessments_from_mongodb(
                model_id=current_model_id,
                grid_type=grid_type,
                start_time=start_time,
                end_time=end_time
            )
            
            progress_indicator.value = 50
            
            if not quality_assessments_data:
                status_display.value = "<b>Status:</b> <span style='color: orange;'>⚠️ No quality assessments found</span>"
                progress_indicator.layout.display = 'none'
                return
            
            # Build time series DataFrame
            time_series_data = pd.DataFrame(quality_assessments_data)
            
            # Get latest data point
            if len(time_series_data) > 0:
                latest_data = time_series_data.iloc[-1]
                
                dashboard_data = {
                    'overall_score': latest_data.get('overall_score', 0.0),
                    'completeness': latest_data.get('completeness', 0.0),
                    'coverage': latest_data.get('coverage', 0.0),
                    'consistency': latest_data.get('consistency', 0.0),
                    'accuracy': latest_data.get('accuracy', 0.0),
                    'reliability': latest_data.get('reliability', 0.0),
                    'snr': latest_data.get('snr', 0.0),
                    'uncertainty': latest_data.get('uncertainty', 0.0),
                    'confidence': latest_data.get('confidence', 0.0),
                    'alignment_accuracy': latest_data.get('alignment_accuracy', 0.0),
                    'gap_count': latest_data.get('gap_count', 0),
                    'gap_percentage': latest_data.get('gap_percentage', 0.0)
                }
            else:
                dashboard_data = {}
            
            progress_indicator.value = 80
        else:
            # Use sample data
            start_time = datetime.strptime(time_start.value, '%Y-%m-%d %H:%M')
            end_time = datetime.strptime(time_end.value, '%Y-%m-%d %H:%M')
            
            time_series_data = generate_time_series_quality_data(start_time, end_time)
            progress_indicator.value = 40
            
            # Get latest data point
            latest_data = time_series_data.iloc[-1]
            
            dashboard_data = {
                'overall_score': latest_data['overall_quality'],
                'completeness': latest_data['completeness'],
                'coverage': latest_data['coverage'],
                'consistency': latest_data['consistency'],
                'snr': latest_data['snr']
            }
            
            progress_indicator.value = 80
        
        # Update displays
        update_status_display()
        update_visualizations()
        
        progress_indicator.value = 100
        current_timestamp = datetime.now()
        status_display.value = "<b>Status:</b> <span style='color: green;'>✅ Dashboard updated</span>"
        last_refresh_display.value = f"<b>Last Refresh:</b> {current_timestamp.strftime('%H:%M:%S')}"
        
        # Hide progress after a moment
        time.sleep(0.5)
        progress_indicator.layout.display = 'none'
        progress_indicator.value = 0
        
    except Exception as e:
        status_display.value = f"<b>Status:</b> <span style='color: red;'>❌ Error: {str(e)}</span>"
        progress_indicator.layout.display = 'none'
        import traceback
        traceback.print_exc()

def update_status_display():
    """Update status and statistics displays."""
    global dashboard_data, time_series_data
    
    if not dashboard_data:
        return
    
    # Overall score
    overall_score = dashboard_data.get('overall_score', 0.0)
    color = 'green' if overall_score >= 0.8 else 'orange' if overall_score >= 0.6 else 'red'
    overall_score_display.value = f"<h2 style='color: {color};'>{overall_score:.2f}</h2>"
    
    # Status indicator
    if overall_score >= 0.8:
        status_indicator.value = "<p style='color: green;'>✅ <b>Pass</b></p>"
    elif overall_score >= 0.6:
        status_indicator.value = "<p style='color: orange;'>⚠️ <b>Warning</b></p>"
    else:
        status_indicator.value = "<p style='color: red;'>❌ <b>Fail</b></p>"
    
    # Trend analysis
    if len(time_series_data) > 1:
        # Handle both MongoDB data (overall_score) and sample data (overall_quality)
        quality_col = 'overall_score' if 'overall_score' in time_series_data.columns else 'overall_quality'
        
        if len(time_series_data) >= 20:
            recent_trend = time_series_data[quality_col].iloc[-10:].mean() - time_series_data[quality_col].iloc[:10].mean()
        else:
            # For shorter series, compare first half to second half
            mid = len(time_series_data) // 2
            recent_trend = time_series_data[quality_col].iloc[mid:].mean() - time_series_data[quality_col].iloc[:mid].mean()
        
        if recent_trend > 0.01:
            trend_dir = "<span style='color: green;'>Up</span>"
        elif recent_trend < -0.01:
            trend_dir = "<span style='color: orange;'>Down</span>"
        else:
            trend_dir = "<span style='color: blue;'>Stable</span>"
        
        trend_direction.value = f"<p><b>Direction:</b> {trend_dir}</p>"
        trend_magnitude.value = f"<p><b>Magnitude:</b> {recent_trend:.3f}</p>"
        
        # Calculate period
        if 'timestamp' in time_series_data.columns:
            time_diff = time_series_data['timestamp'].iloc[-1] - time_series_data['timestamp'].iloc[0]
            days = time_diff.days if hasattr(time_diff, 'days') else (time_diff.total_seconds() / 86400)
            trend_period.value = f"<p><b>Period:</b> {days:.1f} days</p>"
        else:
            trend_period.value = f"<p><b>Period:</b> {len(time_series_data)} assessments</p>"
    
    # Alerts
    alerts = []
    if overall_score < quality_threshold.value:
        alerts.append("⚠️ Quality below threshold")
    if dashboard_data.get('snr', 100) < 15:
        alerts.append("⚠️ SNR decreasing")
    
    alert_count.value = f"<p><b>Active Alerts:</b> <span style='color: orange;'>{len(alerts)}</span></p>"
    if alerts:
        alert_list.value = "<ul>" + "".join([f"<li>{a}</li>" for a in alerts]) + "</ul>"
    else:
        alert_list.value = "<p>No active alerts</p>"
    
    alert_display.value = f"<b>Alerts:</b> {len(alerts)} active warnings"

def update_visualizations():
    """Update all visualizations."""
    global dashboard_data, time_series_data
    
    # Main quality chart
    with main_chart_output:
        clear_output(wait=True)
        
        if len(time_series_data) == 0:
            display(HTML("<p>No data available</p>"))
            return
        
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Handle both MongoDB data and sample data
        timestamp_col = 'timestamp' if 'timestamp' in time_series_data.columns else time_series_data.index
        quality_col = 'overall_score' if 'overall_score' in time_series_data.columns else 'overall_quality'
        
        # Plot selected metrics
        if metric_data_quality.value:
            ax.plot(timestamp_col, time_series_data[quality_col], 
                   label='Overall Quality', linewidth=2)
        if metric_completeness.value and 'completeness' in time_series_data.columns:
            ax.plot(timestamp_col, time_series_data['completeness'], 
                   label='Completeness', linewidth=2, linestyle='--')
        if metric_data_quality.value and 'coverage' in time_series_data.columns:
            ax.plot(timestamp_col, time_series_data['coverage'], 
                   label='Coverage', linewidth=2, linestyle=':')
        if metric_signal_quality.value and 'snr' in time_series_data.columns:
            # Normalize SNR for display (0-1 range)
            snr_normalized = (time_series_data['snr'] - time_series_data['snr'].min()) / (time_series_data['snr'].max() - time_series_data['snr'].min() + 1e-10)
            ax.plot(timestamp_col, snr_normalized, 
                   label='SNR (normalized)', linewidth=2, linestyle='-.')
        
        ax.axhline(y=quality_threshold.value, color='r', linestyle='--', alpha=0.5, label=f'Threshold ({quality_threshold.value:.2f})')
        ax.set_xlabel('Time')
        ax.set_ylabel('Quality Score')
        ax.set_title('Quality Metrics Over Time')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    
    # Quality map (2D slice visualization)
    with quality_map_output:
        clear_output(wait=True)
        
        if dashboard_data:
            fig, ax = plt.subplots(figsize=(10, 6))
            
            # Create a simple quality map (simplified)
            quality_map = np.random.random((50, 50)) * 0.3 + dashboard_data.get('overall_score', 0.7)
            quality_map = np.clip(quality_map, 0.0, 1.0)
            
            im = ax.imshow(quality_map, cmap='RdYlGn', origin='lower', vmin=0, vmax=1)
            ax.set_title('Quality Map (2D Slice)')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            plt.colorbar(im, ax=ax, label='Quality Score')
            plt.tight_layout()
            plt.show()
    
    # Metrics grid
    with metrics_grid_output:
        clear_output(wait=True)
        
        if dashboard_data:
            fig, axes = plt.subplots(1, 4, figsize=(14, 3))
            
            metrics_to_show = [
                ('Overall', dashboard_data.get('overall_score', 0)),
                ('Completeness', dashboard_data.get('completeness', 0)),
                ('Coverage', dashboard_data.get('coverage', 0)),
                ('Consistency', dashboard_data.get('consistency', 0))
            ]
            
            for idx, (name, value) in enumerate(metrics_to_show):
                color = 'green' if value >= 0.8 else 'orange' if value >= 0.6 else 'red'
                axes[idx].barh([0], [value], color=color, height=0.5)
                axes[idx].set_xlim(0, 1)
                axes[idx].set_title(name)
                axes[idx].set_xlabel('Score')
                axes[idx].set_yticks([])
                axes[idx].text(value + 0.05, 0, f'{value:.2f}', va='center')
            
            plt.tight_layout()
            plt.show()
    
    # Comparison view (when in comparison mode)
    with comparison_output:
        clear_output(wait=True)
        
        if dashboard_mode.value == 'comparison':
            fig, axes = plt.subplots(1, 2, figsize=(14, 5))
            
            # Generate comparison data
            comp_data1 = generate_time_series_quality_data(
                datetime.now() - timedelta(days=14),
                datetime.now() - timedelta(days=7),
                5
            )
            comp_data2 = generate_time_series_quality_data(
                datetime.now() - timedelta(days=7),
                datetime.now(),
                5
            )
            
            axes[0].plot(comp_data1['timestamp'], comp_data1['overall_quality'], 
                        label='Period 1', linewidth=2)
            axes[0].plot(comp_data2['timestamp'], comp_data2['overall_quality'], 
                        label='Period 2', linewidth=2)
            axes[0].set_xlabel('Time')
            axes[0].set_ylabel('Quality Score')
            axes[0].set_title('Quality Comparison')
            axes[0].legend()
            axes[0].grid(True, alpha=0.3)
            
            # Difference plot
            diff = comp_data2['overall_quality'].mean() - comp_data1['overall_quality'].mean()
            axes[1].bar(['Difference'], [diff], color='green' if diff > 0 else 'red')
            axes[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
            axes[1].set_ylabel('Quality Change')
            axes[1].set_title('Statistical Comparison')
            axes[1].grid(True, alpha=0.3, axis='y')
            
            plt.tight_layout()
            plt.show()
        else:
            display(HTML("<p>Comparison view available in Comparison mode</p>"))

# Connect events
# Function to update UI based on data source mode
def update_data_source_mode(change):
    """Show/hide MongoDB widgets based on data source mode."""
    if change['new'] == 'mongodb':
        model_selector.layout.display = 'flex'
        grid_type_filter.layout.display = 'flex'
    else:
        model_selector.layout.display = 'none'
        grid_type_filter.layout.display = 'none'

# Connect events
data_source_mode.observe(update_data_source_mode, names='value')
update_data_source_mode({'new': data_source_mode.value})
refresh_button.on_click(refresh_dashboard)
dashboard_mode.observe(lambda x: update_visualizations(), names='value')
chart_type.observe(lambda x: update_visualizations(), names='value')
metric_data_quality.observe(lambda x: update_visualizations(), names='value')
metric_completeness.observe(lambda x: update_visualizations(), names='value')
quality_threshold.observe(lambda x: update_visualizations(), names='value')

# Initial refresh
refresh_dashboard()

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Summary

Congratulations! You've learned how to create interactive quality dashboards.

### Key Takeaways

1. **Dashboard Modes**: Real-time, historical, and comparison modes for different use cases
2. **Time Series Visualization**: Interactive charts showing quality trends over time
3. **Metrics Grid**: Multiple quality metrics displayed as gauge charts
4. **Trend Analysis**: Automatic trend detection and direction indicators
5. **Alert System**: Real-time alerts for quality threshold violations

### Next Steps

Proceed to:
- **09_Statistical_Analysis.ipynb** - Learn statistical analysis methods
- **10_Anomaly_Detection.ipynb** - Learn anomaly detection techniques

### Related Resources

- Quality Dashboard Documentation: `../docs/AM_QADF/05-modules/quality.md`
- API Reference: `../docs/AM_QADF/06-api-reference/quality-api.md`
- Examples: `../examples/`
